## INF1608 - Análise Numérica - 2016.1
## Departamento de Informática - PUC-Rio 
## Prof. Hélio Lopes - lopes@inf.puc-rio.br
## http://www.inf.puc-rio.br/~lopes



## Lista 1

1) Modifique a implementação da decomposição LU para incluir a permutação de linhas, obtendo a decomposição PA=LU.

2) Implemente a função LUsolve que resolve o sistema Ax=b, tendo em mão a deomposição PA=LU.

3) Utilize as implementações dos items 1 e 2 da lista para resolver o sistema:

A = np.matrix([[1.,0.,0.,1.],[-1.,1.,0.,1.],[-1.,-1.,1.,1.],[-1.,-1.,-1.,1.]])
e
b = np.array([1.,1.,2.,0.])

4) Faça uma função que verique se uma matriz A de tamanho nxn é estritamente diagonal dominante:
        Definição: Uma matriz A (nxn) é estritamente diagonal dominante se para toda linha i vale:
                   $$|A_{i,i}| > \sum_{i=1, i\ne j}^n |A_{i,j}|$$
                   
5) Implemente o método de Jacobi para a solução de um sistema de equações lineares. Defina um critério de parada para o seu método, e explique-o. 
        https://pt.wikipedia.org/wiki/M%C3%A9todo_de_Jacobi

6) Implemente o método de Gauss-Seidel para a solução de um sistema de equações lineares. Defina um critério de parada para o seu método, e explique-o.
        https://pt.wikipedia.org/wiki/M%C3%A9todo_de_Gauss-Seidel

7) Teste muito bem todos esses algoritmos!

In [12]:
### Questão Número 1 ###
import numpy as np

# Função que retorna o índice da linha com o maior pivô disponível
# de uma matriz de acordo com uma coluna.
# A: matriz que terá o pivô procurado
# c: índice da coluna que terá o pivô procurado
def FindMaxPivotLineIndex(A,c):
    
    maxPivot = np.absolute(A[c][c])
    maxPivotLineIndex = c
    
    n = len(A)
    for i in range(c,n):
        
        if np.absolute(A[i][c]) > maxPivot:
            maxPivot = np.absolute(A[i][c])
            maxPivotLineIndex = i        
    
    return maxPivotLineIndex

# Função que retorna uma matriz de permutação.
# n: dimenção da matriz quadrada
# l1: índice da primeira linha que será trocada com a segunda
# l2: índice da segunda linha que será trocada com a primeira
def BuildPermutationMatrix(n,l1,l2):
    
    P = np.identity(n)
    
    if l1 >= n or l2 >= n:
        return P
    
    P[l1][l1] = 0.0
    P[l1][l2] = 1.0
    
    P[l2][l2] = 0.0
    P[l2][l1] = 1.0
    
    return P

# Decomposição LU com permutação de linhas.
# A: matriz que será decomposta
def LUdecompWithPermutation(A):
    
    LU = np.copy(A)
    n = len(A)
    vetP = np.zeros((n-1,n,n))
    
    for j in range(n-1):
        
        pivotLineIndex = FindMaxPivotLineIndex(LU,j)
        auxP = BuildPermutationMatrix(n,j,pivotLineIndex)
        LU = np.dot(auxP,LU)
        vetP[j] = auxP
        
        for i in range(j+1,n):            
            LU[i][j] = LU[i][j]/LU[j][j]
            for k in range(j+1,n):
                LU[i][k] = LU[i][k] - LU[i][j]*LU[j][k]

    P = reduce(np.dot, vetP[::-1])
    return np.array((P,LU))

### Questão Número 2 ###
import numpy as np

def LUdecomp(A):
    LU = np.copy(A)
    n = len(A)
    for j in range(n-1):
        for i in range(j+1,n):
            LU[i][j] = LU[i][j]/LU[j][j]
            for k in range(j+1,n):
                LU[i][k] = LU[i][k] - LU[i][j]*LU[j][k]
    return LU

def LUforwardsub (L,b):
    n = len(L)
    y = np.zeros(n)
    y[0] = b[0]
    for i in range(1,n):
        y[i] = b[i]
        for j in range(i):
            y[i] = y[i] - L[i][j]*y[j]
    return y
            
def LUbackwardsub (U,y):
    n = len(U)
    x = np.zeros(n)
    x[n-1] = y[n-1]/U[n-1,n-1]
    for i in range(n-2,-1,-1):
        x[i] = y[i]
        for j in range(i+1,n):
            x[i] = x[i] - U[i][j]*x[j]
        x[i] /= U[i][i]
    return x

def LUsolveWithPermutation(P,LU,b):
    b = np.dot(P,b)
    y = LUforwardsub(LU,b)
    x = LUbackwardsub(LU,y)
    return x
    
def LUsolve(LU,b):
    y = LUforwardsub(LU,b)
    x = LUbackwardsub(LU,y)
    return x

#A = np.matrix([[1.,0.,0.,1.],[-1.,1.,0.,1.],[-2.,-1.,1.,1.],[-1.,-1.,-1.,1.]])
A = np.matrix([[3.0,-4.0,1.0],[1.0,2.0,2.0],[4.0,0.0,-3.0]])
#b = np.array([1.,1.,2.,0.])
b = np.array([9.0,3.0,-2.0])
print(A)
print(b)

print("\n")

PAndLU = LUdecompWithPermutation(A)
print(PAndLU)
print(LUsolveWithPermutation(PAndLU[0],PAndLU[1],b))

print("\n")

LU = LUdecomp(A)
print(LU)
print(LUsolve(LU,b))

[[ 3. -4.  1.]
 [ 1.  2.  2.]
 [ 4.  0. -3.]]
[ 9.  3. -2.]


[[[ 0.     0.     1.   ]
  [ 1.     0.     0.   ]
  [ 0.     1.     0.   ]]

 [[ 4.     0.    -3.   ]
  [ 0.75  -4.     3.25 ]
  [ 0.25  -0.5    4.375]]]
[ 1. -1.  2.]


[[ 3.         -4.          1.        ]
 [ 0.33333333  3.33333333  1.66666667]
 [ 1.33333333  1.6        -7.        ]]
[ 1. -1.  2.]


In [31]:
### Questão Número 4 ###
import numpy as np

def IsEstritamenteDiagonal(A):    
    M = np.copy(A)
    for i in range(len(M)):        
        acum = 0.0
        for j in range(len(M)):            
            if i != j:
                acum = acum + np.absolute(M[i][j])
                
        if acum >= M[i][i]:
            return False
        
    return True

A = np.matrix(([1.,0.,0.],[1.,4.1,3.],[0.,0.,1.]))
print(IsEstritamenteDiagonal(A))

True
